# Setup Ports

In [1]:
import sys
from pathlib import Path
project_root = Path.cwd().parent.parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))


from src.maritime_module.utils.port_utils import *

port = PortData()

## List of Ports

In [2]:
port.get_port_names()

['AASIAAT',
 'ABADAN',
 'ABASHIRI KO',
 'ABENRA',
 'ABERDEEN',
 'ABERDEEN',
 'ABIDJAN',
 'ABU KHAMMASH',
 'ABU ZABY',
 'ACAJUTLA',
 'ACAJUTLA OFFSHORE TERMINAL',
 'ACAPULCO',
 'ACHLADI',
 'AD DAKHLA',
 'ADAK NAVAL AIR STATION',
 'ADEN',
 'ADLER',
 'ADMIRALTY BAY',
 'ADRA',
 'AEROSKOBING',
 'AGADIR',
 'AGBAMI OIL TERMINAL',
 'AGUADILLA',
 'AGUILAS',
 'AHOUSAT',
 'AHUKINI LANDING',
 'AHUS',
 'AIGIO',
 'AIN SUKHNA TERMINAL',
 'AIOI',
 'AJMAN',
 'AKAROA',
 'AKITA-FUNAKAWA KO',
 'AKPO OIL TERMINAL',
 'AKRA KAVONISI',
 'AKRAHAMN',
 'AKSAZ LIMANI',
 'AKUREYRI',
 'AKUTAN HARBOR',
 'AL AHMADI',
 'AL AQABAH',
 'AL BASRAH',
 'AL BURAYQAH',
 'AL HAMRIYAH LPG TERMINAL',
 'AL ISKANDARIYH (ALEXANDRIA)',
 'AL JAZEERA PORT',
 'AL JUBAYL',
 'AL KHAIR OIL TERMINAL',
 'AL KUWAYT',
 'AL LADHIQIYAH',
 'AL MANAMAH',
 'AL MUKALLA',
 'AL MUKHA',
 'AL QUSAYR',
 'AL RAYYAN TERMINAL',
 'AL SHAHEEN TERMINAL',
 'AL-BASRA OIL TERMINAL',
 'ALABASTER',
 'ALAMEDA',
 'ALANYA',
 'ALBANY',
 'ALBANY',
 'ALBORG',
 'ALCAN HA

## Port Series

In [3]:
port_name = "LOs angeles"
idx = port.get_port_index(port_name)
port_series = port.get_port_by_name(port_name)
print(idx)
port_series

1673


INDEX_NO                    16080.0
REGION_NO                   16000.0
PORT_NAME               LOS ANGELES
COUNTRY                          US
LATITUDE                      33.75
                      ...          
ENG_SUPPLY                        Y
REPAIRCODE                        A
DRYDOCK                           M
RAILWAY                           M
geometry      POINT (-118.25 33.75)
Name: 1014, Length: 79, dtype: object

## Port String with Position

In [4]:
port.format_port_string(port_series)

"LOS ANGELES, US (LAT: 33° 45.0' N  LONG: 118° 15.0' W)"

## Cleaned Series into Dataframe

In [5]:
port.get_port_details_df(port_series)

,Value
PORT_NAME,LOS ANGELES
COUNTRY,US
LATITUDE,33.75
LONGITUDE,-118.25
PUB,CP07
CHART,18751
HARBORSIZE,L
HARBORTYPE,CB
SHELTER,E
ENTRYOTHER,Y


# Boudaries

In [14]:
port1 = port.get_port_by_name("Los Angeles")
port2 = port.get_port_by_name("San Francisco")
port3 = port.get_port_by_name("Miami")
print(port1.geometry)
bbox = Boundaries()
bbox.create_geo_boundary(geometries = [port1.geometry, port2.geometry, port3.geometry])

POINT (-118.25 33.75)


,geometry
0,"POLYGON ((-80.183 25.783, -80.183 37.817, -122..."
